In [3]:
import pandas as pd
from pandas_datareader import data
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import GridSearchCV

In [6]:
# Load the California housing dataset
california = fetch_california_housing()

# Create a DataFrame with the features
df = pd.DataFrame(california.data, columns=california.feature_names)

# Add the target variable as a new column
df['MedHouseVal'] = california.target

# Display the first few rows of the DataFrame
df.head()


MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  MedHouseVal  
0    -122.23        4.526  
1    -122.22        3.585  
2    -122.24        3.521  
3    -122.25        3.413  
4    -122.25        3.422

In [8]:
# Assuming df is already created as in the previous steps
X = df.iloc[:, :-1]  # Select all columns except the last one as features
y = df.iloc[:, -1]   # Select the last column as the target variable

# Display the shapes of X and y to verify
print(X.shape)
print(y.shape)


(20640, 8)
(20640,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)


In [12]:
rt = DecisionTreeRegressor(criterion = 'squared_error', max_depth=5)


In [13]:
rt.fit(X_train,y_train)


DecisionTreeRegressor(max_depth=5)

In [14]:
y_pred = rt.predict(X_test)


In [15]:
r2_score(y_test,y_pred)


0.5997321244428706

## Hyperparameter Tuning

In [16]:
# Define the parameter grid
param_grid = {
    'max_depth': [2, 4, 8, 10, None],
    'criterion': ['squared_error', 'absolute_error'],  # Updated criterion names
    'max_features': [0.25, 0.5, 1.0],
    'min_samples_split': [2, 5, 10]  # Typically, this is an integer value
}

In [17]:
# Initialize the DecisionTreeRegressor
dt_regressor = DecisionTreeRegressor()

In [18]:
# Set up the GridSearchCV with joblib to speed up by using all the cores
grid_search = GridSearchCV(estimator=dt_regressor, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)


In [19]:
# Fit the model
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error'],
                         'max_depth': [2, 4, 8, 10, None],
                         'max_features': [0.25, 0.5, 1.0],
                         'min_samples_split': [2, 5, 10]},
             verbose=2)

In [20]:
# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'criterion': 'squared_error', 'max_depth': 8, 'max_features': 1.0, 'min_samples_split': 10}
Best Score: 0.6985207739538455


## Feature Importance 

In [22]:
# Assuming you've already fitted your model, for example with:
best_model = grid_search.best_estimator_

In [23]:
# Get feature importances
feature_importances = best_model.feature_importances_

In [24]:
# Print feature importances along with feature names
for importance, name in sorted(zip(feature_importances, X_train.columns), reverse=True):
    print(f"{name}: {importance:.4f}")


MedInc: 0.6645
AveOccup: 0.1323
Latitude: 0.0614
Longitude: 0.0504
HouseAge: 0.0421
AveRooms: 0.0328
AveBedrms: 0.0089
Population: 0.0076
